In [2]:
import sys
sys.path.append('/home/td/Documents/sports_predictor/nba')
sys.path.append('/home/td/Documents/sports_predictor')

import pandas as pd
import lightgbm
import pandas
from sklearn import metrics, model_selection
from nba.common import (
    timeit,
)
import pre
from nba.data_pipeline import load_general_feature_file


In [3]:

class Model():
    max_iter = 1000000

    lightgbm_max_iter = 10000
    lightgbm_early_stopping_rounds = 100

    def __init__(self, model_type, model_params):
        self.model_type = model_type
        self.model_params = model_params
        self.transformers_dict = dict()

    @timeit
    def fit(self, x, y):
        print('entered fit, x shape: {}'.format(x.shape))
        self.transformers_dict = dict()
        self.columns = x.columns

        if self.model_type == 'lightgbm':
            x_train, x_val, y_train, y_val = model_selection.train_test_split(x, y)
            lgtrain = lightgbm.Dataset(x_train, y_train)
            lgvalid = lightgbm.Dataset(x_val, y_val)

            self.model = lightgbm.train(
                self.model_params,
                lgtrain,
                num_boost_round=self.lightgbm_max_iter,
                valid_sets=[lgtrain, lgvalid],
                valid_names=['train', 'valid'],
                early_stopping_rounds=self.lightgbm_early_stopping_rounds,
                verbose_eval=100
            )

    @timeit
    def predict(self, x):
        if self.model_type == 'lightgbm':
            preds = self.model.predict(x, num_iteration=self.model.best_iteration)
            preds = np.rint(preds).astype(int)
            return preds

    def get_metric(self, x, y):
        preds = self.predict(x)
        return metrics.accuracy_score(y, preds)

    def evaluate(self):
        if self.model_type == 'lightgbm':
            output = []

            for i, j in zip(self.columns, self.model.feature_importance('gain', iteration=self.model.best_iteration)):
                output.append({'column': i, 'feature_importance': j})
            return pd.DataFrame.from_dict(output).sort_values('feature_importance', ascending=False)
        

In [7]:
feature_df = load_general_feature_file()
x = feature_df.drop('win', axis = 1)
y = feature_df['win']

In [8]:
lgbm_params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_error',
    "learning_rate": 0.01,
    "max_depth": -1,
    'num_leaves': 31,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    'bagging_freq': 1,
}
model = Model('lightgbm', lgbm_params)
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y)
model.fit(x_train, y_train)
model.get_metric(x_test, y_test)

function: 'fit' starting
entered fit, x shape: (2964, 218)
Training until validation scores don't improve for 100 rounds
[100]	train's binary_error: 0.168241	valid's binary_error: 0.385965
Early stopping, best iteration is:
[37]	train's binary_error: 0.220423	valid's binary_error: 0.357625
function: 'fit' finished in  0.40 seconds
function: 'predict' starting
function: 'predict' finished in  0.03 seconds


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [5]:
model.evaluate()

AttributeError: 'Model' object has no attribute 'columns'